Imports

In [18]:
import xml.etree.ElementTree as ET
import json
from collections import defaultdict

Helper functions

In [19]:
from collections import defaultdict


def build_element_tree(root):
    def recursive_build(element):
        if element.tag != 'Item':
            return None
        
        item_id = element.find('ID').text
        name = element.find('Name').text
        description = element.find('Description').text
        
        children = []
        for child in element.find('Children') or []:
            child_tree = recursive_build(child)
            if child_tree:
                children.append(child_tree)
        
        return {
            'id': item_id,
            'children': children,
            'properties': set()
        }

    tree = []
    for item in root.find('.//Items'):
        item_tree = recursive_build(item)
        if item_tree:
            tree.append(item_tree)
    
    return tree

def get_properties(root):
    properties = defaultdict(set)
    for prop_def in root.findall('.//PropertyDefinition'):
        name_elem = prop_def.find('Name')
        if name_elem is not None:
            prop_name = name_elem.text
            for class_id in prop_def.findall('.//ClassificationID/ItemID'):
                if class_id is not None:
                    item_id = class_id.text
                    properties[item_id].add(prop_name)
    return properties

def assign_properties(tree, properties):
    for node in tree:
        if node['id'] in properties:
            node['properties'] = properties[node['id']]
        assign_properties(node['children'], properties)

def sets_to_lists(obj):
    if isinstance(obj, dict):
        return {key: sets_to_lists(value) for key, value in obj.items()}
    elif isinstance(obj, set):
        return list(obj)
    else:
        return obj

Create tree structure

In [20]:
# Load the XML file
tree = ET.parse('input.xml')
root = tree.getroot()

# Build the element tree and get properties
element_tree = build_element_tree(root)
print("Element tree created.")

properties = get_properties(root)
assign_properties(element_tree, properties)
print("Properties assigned to element tree.")
with open('entry_tree.json', 'w') as f:
    json.dump(element_tree, f, indent=2, default=lambda x: list(x) if isinstance(x, set) else x)

Element tree created.
Properties assigned to element tree.


C:\Users\CDRI\AppData\Local\Temp\ipykernel_17728\3392539100.py:14: DeprecationWarning: Testing an element's truth value will raise an exception in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  for child in element.find('Children') or []:


Export existing config (testing purposes)

In [21]:
def export_config_prev(element_tree):
    def recursive_export(node, parent_properties=None):
        if parent_properties is None:
            parent_properties = set()

        config_node = {
            'id': node['id']
        }

        not_inherited = parent_properties - node['properties']
        if not_inherited and len(node['properties']) > 0:
            config_node['not_inherited_from'] = not_inherited

        new_props = node['properties'] - parent_properties
        if new_props and node['properties'] != parent_properties:
            config_node['new_properties'] = new_props

        # Process children
        all_child_properties = set()
        children = []
        for child in node['children']:
            child_config = recursive_export(child, node['properties'])
            if child_config:
                children.append(child_config)
                all_child_properties |= child_config.get('new_properties', set())
                all_child_properties |= (node['properties'] - child_config.get('not_inherited_from', set()))

        if children:
            config_node['children'] = children

        # Calculate properties that are never inherited to any child
        if node['children']:
            never_inherit = node['properties'] - all_child_properties
            if never_inherit:
                config_node['never_inherit_to'] = never_inherit

        return config_node if len(config_node) > 0 else None

    config_prev = []
    for root_node in element_tree:
        root_config = recursive_export(root_node)
        if root_config:
            config_prev.append(root_config)

    return config_prev

# Usage
config_prev = export_config_prev(element_tree)

# Convert sets to lists for JSON serialization and remove empty lists
def clean_and_convert(obj):
    if isinstance(obj, dict):
        return {k: clean_and_convert(v) for k, v in obj.items() if v}
    elif isinstance(obj, list):
        return [clean_and_convert(v) for v in obj if v]
    elif isinstance(obj, set):
        return list(obj) if obj else None
    else:
        return obj

serializable_config = clean_and_convert(config_prev)

print("Existing configuration extracted:")
print(json.dumps(serializable_config, indent=2))

# Save config_prev to a file
with open('config_prev.json', 'w') as f:
    json.dump(serializable_config, f, indent=2)

Existing configuration extracted:
[
  {
    "id": "Site",
    "children": [
      {
        "id": "Geographic Element",
        "children": [
          {
            "id": "Terrain"
          },
          {
            "id": "Site Geometry"
          },
          {
            "id": "Flora & Fauna"
          }
        ]
      },
      {
        "id": "Massing",
        "children": [
          {
            "id": "Morph"
          }
        ]
      },
      {
        "id": "Civil Element"
      }
    ]
  },
  {
    "id": "Space",
    "new_properties": [
      "Fire exit space",
      "Finishing - Floors",
      "Finishing - Walls",
      "Access - accessibility for the disabled",
      "Wall finish thickness (requirement)",
      "Risk factor",
      "Usage type of space",
      "Status (Space and Zone)",
      "Number of expected occupants",
      "Phasing",
      "Sprinkling",
      "Documentation",
      "Finishing - Ceilings",
      "Access - Accessible to the public",
      "Finish

Process config and apply

In [22]:
def apply_new_config(element_tree, new_config):
    def find_config_node(config, node_id):
        for item in config:
            if item['id'] == node_id:
                return item
            if 'children' in item:
                result = find_config_node(item['children'], node_id)
                if result:
                    return result
        return None

    def process_node(node, parent_properties=None):
        if parent_properties is None:
            parent_properties = set()

        # Find the corresponding config for this node
        config_node = find_config_node(new_config, node['id'])

        # Start with parent properties
        node['properties'] = set(parent_properties)

        if config_node:
            # Remove properties that should not be inherited
            if 'not_inherited_from' in config_node:
                node['properties'] -= set(config_node['not_inherited_from'])

            # Add new properties
            if 'new_properties' in config_node:
                node['properties'] |= set(config_node['new_properties'])

        # Process children after the current node is fully processed
        for child in node['children']:
            process_node(child, node['properties'])

        # Remove properties that should never be inherited to children
        if config_node and 'never_inherit_to' in config_node:
            for child in node['children']:
                child['properties'] -= set(config_node['never_inherit_to'])

    # Process each root node
    for root_node in element_tree:
        process_node(root_node)

# Load the new configuration
with open('config.json', 'r') as f:
    new_config = json.load(f)

# Apply the new configuration to the element tree
apply_new_config(element_tree, new_config)

print("New configuration applied to element tree.")

# Function to convert sets to lists for JSON serialization
def sets_to_lists(obj):
    if isinstance(obj, dict):
        return {k: sets_to_lists(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [sets_to_lists(v) for v in obj]
    elif isinstance(obj, set):
        return list(obj)
    else:
        return obj

# Print the updated element tree
print(json.dumps(sets_to_lists(element_tree), indent=2))

# Optionally, save the updated element tree to a file
with open('updated_element_tree.json', 'w') as f:
    json.dump(sets_to_lists(element_tree), f, indent=2)

New configuration applied to element tree.
[
  {
    "id": "Site",
    "children": [
      {
        "id": "Geographic Element",
        "children": [
          {
            "id": "Terrain",
            "children": [],
            "properties": []
          },
          {
            "id": "Site Geometry",
            "children": [],
            "properties": []
          },
          {
            "id": "Flora & Fauna",
            "children": [],
            "properties": []
          }
        ],
        "properties": []
      },
      {
        "id": "Massing",
        "children": [
          {
            "id": "Morph",
            "children": [],
            "properties": []
          }
        ],
        "properties": []
      },
      {
        "id": "Civil Element",
        "children": [],
        "properties": []
      }
    ],
    "properties": []
  },
  {
    "id": "Space",
    "children": [
      {
        "id": "External Space",
        "children": [
          {
        

Update and save XML

In [23]:
try:
    from lxml import ET
except ImportError:
    import xml.etree.ElementTree as ET

def update_xml_properties(original_xml_path, updated_element_tree, output_xml_path):
    # Parse the original XML
    tree = ET.parse(original_xml_path)
    root = tree.getroot()

    # Find all PropertyDefinitionGroup elements
    prop_def_groups = root.findall('.//PropertyDefinitionGroup')
    if not prop_def_groups:
        print("Error: No PropertyDefinitionGroups found in the XML.")
        return

    # Helper function to find a node in the updated_element_tree
    def find_node(tree, node_id):
        for node in tree:
            if node['id'] == node_id:
                return node
            if 'children' in node:
                result = find_node(node['children'], node_id)
                if result:
                    return result
        return None

    # Collect all properties and their associated class IDs
    property_classes = {}
    def collect_properties(node):
        for prop in node['properties']:
            if prop not in property_classes:
                property_classes[prop] = set()
            property_classes[prop].add(node['id'])
        for child in node.get('children', []):
            collect_properties(child)

    for root_node in updated_element_tree:
        collect_properties(root_node)

    # Update existing PropertyDefinitions or create new ones
    for prop_def_group in prop_def_groups:
        prop_defs = prop_def_group.find('PropertyDefinitions')
        if prop_defs is None:
            prop_defs = ET.SubElement(prop_def_group, 'PropertyDefinitions')

        # Update existing PropertyDefinitions
        for prop_def in prop_defs.findall('PropertyDefinition'):
            prop_name = prop_def.find('Name').text
            if prop_name in property_classes:
                # Update ClassificationIDs
                class_ids_elem = prop_def.find('ClassificationIDs')
                if class_ids_elem is None:
                    class_ids_elem = ET.SubElement(prop_def, 'ClassificationIDs')
                else:
                    class_ids_elem.clear()

                for class_id in property_classes[prop_name]:
                    class_id_elem = ET.SubElement(class_ids_elem, 'ClassificationID')
                    ET.SubElement(class_id_elem, 'ItemID').text = class_id
                    ET.SubElement(class_id_elem, 'SystemIDName').text = 'ARCHICAD Classification'
                    ET.SubElement(class_id_elem, 'SystemIDVersion').text = 'v 2.0'

                # Remove this property from the dictionary as it's been processed
                del property_classes[prop_name]

        # Add new PropertyDefinitions for remaining properties
        for prop_name, class_ids in property_classes.items():
            new_prop_def = ET.SubElement(prop_defs, 'PropertyDefinition')
            ET.SubElement(new_prop_def, 'Name').text = prop_name
            ET.SubElement(new_prop_def, 'Description')
            value_desc = ET.SubElement(new_prop_def, 'ValueDescriptor', Type="SingleValueDescriptor")
            ET.SubElement(value_desc, 'ValueType').text = 'String'
            ET.SubElement(new_prop_def, 'MeasureType').text = 'Default'
            default_value = ET.SubElement(new_prop_def, 'DefaultValue')
            ET.SubElement(default_value, 'DefaultValueType').text = 'Basic'
            variant = ET.SubElement(default_value, 'Variant', Type="StringVariant")
            ET.SubElement(variant, 'Status').text = 'UserUndefined'
            class_ids_elem = ET.SubElement(new_prop_def, 'ClassificationIDs')
            for class_id in class_ids:
                class_id_elem = ET.SubElement(class_ids_elem, 'ClassificationID')
                ET.SubElement(class_id_elem, 'ItemID').text = class_id
                ET.SubElement(class_id_elem, 'SystemIDName').text = 'ARCHICAD Classification'
                ET.SubElement(class_id_elem, 'SystemIDVersion').text = 'v 2.0'

    ET.indent(tree, space="\t", level=0)
    # Save the updated XML
    tree.write(output_xml_path, encoding='UTF-8', xml_declaration=True)

    #formatter = xmlformatter.Formatter(indent="1", indent_char="\t",encoding_input="UTF-8", encoding_output="UTF-8", inline=False)
    #formatter.format_file(output_xml_path)
    
    print(f"Updated XML saved to {output_xml_path}")

# Usage
update_xml_properties('input.xml', element_tree, 'output.xml')

Updated XML saved to output.xml
